## lite helpers

jupyterlite is _magic_, but sometimes needs... _more magic_.

In [ ]:
from __future__ import annotations

from pathlib import Path

## data uris

the `data:` uri scheme lets us inject just about any standalone content into an `iframe`. using `base64` encoding is heavier, but works for text and binary data.

In [ ]:
def data_uri(data: bytes, mime: str) -> str:
    import base64

    data_b64 = base64.b64encode(data).decode("utf-8")
    return f"data:{mime};base64,{data_b64}"

## data uri iframes

the `src` of an `iframe` can be a data uri.

In [ ]:
def data_uri_iframe(data: bytes, width, height):
    from IPython.display import IFrame, display

    display(IFrame(data_uri(data, "text/html"), width=width, height=height))

## inlining assets


In [ ]:
def asset_uris(root_path: Path, globs: dict[str, str] | None = None) -> dict[str, str]:
    assets = {}
    globs = globs or {"*.js": "application/json", "*.css": "text/css", "*.png": "image/png"}
    for glob, mime in globs.items():
        for asset in root_path.rglob(glob):
            assets[str(asset.relative_to(root_path).as_posix())] = data_uri(
                asset.read_bytes(), mime
            )
    return assets

## special cases

### pytest html
`pytest-html` uses some brower APIs not available in `iframe` elements with `src="data:..."`. This wrapper cleans them up enough to _work_, but not all features will be enabled.

In [ ]:
def pytest_html(path, width="100%", height="600px"):
    text = Path(path).read_text(encoding="utf-8")
    text = text.replace("sessionStorage", "fakeSessionStorage").replace(
        "<script>",
        """<script>
            class FakeSessionStorage {
                constructor() { this._data = {}; }
                getItem = (key) => this._data[key] || null;
                setItem = (key, value) => this._data[key] = value;
            }
            window.fakeSessionStorage = new FakeSessionStorage();
        """,
    )
    data_uri_iframe(text.encode(), width, height)

### coverage html

we can rewrite links to web assets with inline URIs, and then re-encode them as `iframe` elements.

In [ ]:
def coverage_html(root, width="100%", height="600px"):
    from pathlib import Path

    root_path = Path(root)
    assets = asset_uris(root_path)

    for path in sorted(root_path.glob("*.html")):
        text = path.read_text(encoding="utf-8")
        for filename, uri in assets.items():
            for attr in ["href", "src"]:
                text = text.replace(f'{attr}="{filename}"', f'{attr}="{uri}"')
        data_uri_iframe(text.encode(), width, height)